<a href="https://colab.research.google.com/github/meg190/aplicadas-II-colabs/blob/main/aplicadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Análisis de transiciones académicas usando cadenas de markov

##Resumen

Comprender la dinámica del rendimiento académico a lo largo del semestre es clave para realizar mejoras pedagógicas y anticipar riesgos en el aprendizaje. Por ello, en este trabajo se modelaron las transiciones entre cuatro niveles de desempeño estudiantil (Básico, En desarrollo, Alto y Sobresaliente) utilizando una cadena de Markov de tiempo discreto. Se construyó la matriz de transición entre evaluaciones, se estimó la distribución estacionaria del sistema y se calcularon probabilidades asociadas a cambios relevantes en el desempeño.Los resultados mostraron que con el paso de las evaluaciones el sistema converge hacia una distribución dominada por niveles medios y altos, y que las probabilidades de descender a estados críticos permanecen bajas. Estas tendencias indican un escenario académico con progresión gradual y estabilidad en el desempeño estudiantil.

##Objetivos
* Estimar la matriz de transición entre niveles académicos a lo largo de las evaluaciones.
* Obtener la distribución estacionaria del sistema.
Calcular probabilidades específicas:  i.  desde Básico → aprobado en 4 evaluaciones ii.  desde Sobresaliente → Básico en 3 evaluaciones.
*Discutir implicaciones pedagógicas y limitaciones.

##Introducción

El rendimiento académico es un indicador fundamental para comprender cómo avanzan los estudiantes a lo largo del tiempo y para orientar intervenciones pedagógicas que fortalezcan el aprendizaje. Desde la psicología educativa, Zimmerman (2002) señala que el desempeño escolar evoluciona dinámicamente conforme los estudiantes se enfrentan a nuevas tareas, regulan su aprendizaje y responden a distintos contextos. Esta perspectiva resalta la necesidad de analizar el rendimiento no como un valor aislado, sino como una trayectoria compuesta por múltiples estados posibles.

Para modelar procesos que evolucionan por etapas y con comportamiento probabilístico, las cadenas de Markov ofrecen un marco teórico adecuado. Feller (1968) describe estos modelos como sistemas estocásticos en los que la transición hacia un estado futuro depende únicamente del estado actual, permitiendo representar fenómenos evolutivos en secuencias discretas. Esta propiedad las convierte en una herramienta útil para estudiar cambios graduales en variables educativas, donde los estudiantes pueden avanzar, mantenerse o retroceder de un estado de rendimiento a otro.

La aplicación de cadenas de Markov al ámbito educativo ha mostrado resultados prometedores en la caracterización de rutas de aprendizaje. Bai y Chang (2019) demostraron que este tipo de modelos permite identificar patrones de progreso académico y estimar riesgos de desempeño bajo, proporcionando información valiosa para la toma de decisiones docentes. Sus hallazgos evidencian que las transiciones entre niveles de rendimiento no son aleatorias, sino que siguen un comportamiento estructurado que puede analizarse a partir de matrices de transición y distribuciones estables.

En este estudio se modelaron las transiciones entre cuatro niveles de rendimiento académico —Básico, En desarrollo, Alto y Sobresaliente— mediante una cadena de Markov de tiempo discreto. El objetivo fue estimar la matriz de transición, determinar la distribución estacionaria y calcular probabilidades de interés con el fin de interpretar la estabilidad del sistema y comprender la evolución del desempeño estudiantil a lo largo del semestre.

En este proyecto se estudiará cómo los estudiantes cambian entre diferentes niveles de rendimiento académico a lo largo de un semestre. Se usará un modelo matemático llamado "Cadenas de Markov" para predecir las probabilidades de que un estudiante pase de un nivel a otro.

Preguntas a resolver:

*   ¿Cómo se mueven los estudiantes entre niveles académicos?
*   ¿Cuál es el estado de equilibrio del sistema? (¿Las notas se mantienen estables?)
* ¿Cuál es la probabilidad de que un estudiante que empieza perdiendo pueda ganar la materia si hay 4 evaluaciones?
* ¿Cuál es la probabilidad de que un estudiante que empezó sobresaliente esté en riesgo de perder después de 3 evaluaciones?

##Marco teorico

Una Cadena de Markov es un modelo matemático que nos dice: "el futuro solo depende del presente, no del pasado". En este caso, significa que la nota que un estudiante obtendrá en la próxima evaluación depende principalmente de su nota actual, no de lo que sacó antes.En este caso los 4 estados academicos son:

* ESTADO 0 - BÁSICO: Notas de 0.0 a 2.9 (En riesgo de perder)
* ESTADO 1 - EN DESARROLLO: Notas de 3.0 a 3.7 (Aprobado pero puede mejorar)
*   ESTADO 2 - ALTO GRADO: Notas de 3.8 a 4.4 (Buen rendimiento)
*  ESTADO 3 - SOBRESALIENTE: Notas de 4.5 a 5.0 (Excelente rendimiento)

##Metodología
Se recolectó la informacion de.... y después se clasificaron las notas de acuerdo a los 4 estados academicos planteados anteriormente para poder realizar una matriz de transición; esta matriz muestra las probabilidades de pasar de un estado a otro, y con la matriz, se puede predecir que pasará después de varias evaluaciones.





Lo que tenemos en los datos son las probabilidades de pasar de un estado de bajo, basico, alto y superior a lo largo de un semestre, donde cada momento representa una evaluacion, y lo ideal es calcular cosas respecto a eso. en python está la matriz de transición que da las probabilidades.

In [1]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/Juanm3005/proyecto-aplicadas-II/refs/heads/main/aplicadas%20proyecto.csv"
df = pd.read_csv(url, sep=";", decimal=",")

def nota_a_estado(n):
    if n <= 2.9:
        return 0  # Básico
    elif 3.0 <= n <= 3.7:
        return 1  # En desarrollo
    elif 3.8 <= n <= 4.4:
        return 2  # Alto grado
    elif 4.5 <= n <= 5.0:
        return 3  # Sobresaliente
    else:
        return np.nan  # si hay valores fuera de rango

# 3. Convertir todas las notas en estados
# Suponiendo que las columnas de notas son algo como "momento1", "momento2", …
cols_notas = [c for c in df.columns if c.startswith("momento")]
df_states = df[cols_notas].map(nota_a_estado)

# 4. Construir matriz de conteos N
n_states = 4
N = np.zeros((n_states, n_states), dtype=int)
for idx, row in df_states.iterrows():
    estados = row.dropna().tolist()
    for a, b in zip(estados[:-1], estados[1:]):
        if not np.isnan(a) and not np.isnan(b):
            N[int(a), int(b)] += 1

# 5. Calcular matriz de transición P (con suavizado, ej. α=1)
alpha = 1.0
Np = N + alpha
row_sums = Np.sum(axis=1, keepdims=True)
P = Np / row_sums

# 6. Mostrar resultados
print("Matriz de conteos N:")
print(N)
print("\nMatriz de transición P:")
print(np.round(P.T, 4))
print("\nSuma de columnas de P:", np.round(P.sum(axis=1), 4))

Matriz de conteos N:
[[32  6  3 13]
 [ 6  3  5 16]
 [ 3  5 12 24]
 [16 15 35 96]]

Matriz de transición P:
[[0.569  0.2059 0.0833 0.1024]
 [0.1207 0.1176 0.125  0.0964]
 [0.069  0.1765 0.2708 0.2169]
 [0.2414 0.5    0.5208 0.5843]]

Suma de columnas de P: [1. 1. 1. 1.]
